In [101]:
using JuMP, Ipopt
using LinearAlgebra
using Combinatorics: combinations
using ForwardDiff: gradient, hessian
using Random

In [94]:
TwoBundles = collect(combinations(1:4, 2))
ThrBundles = collect(combinations(1:4, 3))

TwoOpt = [3, 4]
ThrOpt = [1, 2, 3]

setdiff!(TwoBundles, [TwoOpt])
setdiff!(ThrBundles, [ThrOpt])

display(TwoBundles)
display(ThrBundles)

5-element Vector{Vector{Int64}}:
 [1, 2]
 [1, 3]
 [1, 4]
 [2, 3]
 [2, 4]

3-element Vector{Vector{Int64}}:
 [1, 2, 4]
 [1, 3, 4]
 [2, 3, 4]

In [92]:
mod = Model(Ipopt.Optimizer)
set_silent(mod)

@variable(mod, t[1:4] ≥ 0)
@variable(mod, 0 ≤ f[1:4] ≤ 1)

@constraint(mod, Sortedness[j in 1:3], t[j] ≤ t[j+1])

ε = 0.001

h = 2
@NLconstraint(mod, TwoOptCon[l in 1:length(TwoBundles)],
    sum(t[TwoOpt[i]] * f[TwoOpt[i]] * prod(1 - f[TwoOpt[j]] for j in i+1:h) for i in 1:h-1) + t[TwoOpt[end]] * f[TwoOpt[end]] - ε ≥
    sum(t[TwoBundles[l][i]] * f[TwoBundles[l][i]] * prod(1 - f[TwoBundles[l][j]] for j in i+1:h) for i in 1:h-1) + t[TwoBundles[l][end]] * f[TwoBundles[l][end]]
)
    
h = 3
@NLconstraint(mod, ThrOptCon[l in 1:length(ThrBundles)],
    sum(t[ThrOpt[i]] * f[ThrOpt[i]] * prod(1 - f[ThrOpt[j]] for j in i+1:h) for i in 1:h-1) + t[ThrOpt[end]] * f[ThrOpt[end]] - ε ≥
    sum(t[ThrBundles[l][i]] * f[ThrBundles[l][i]] * prod(1 - f[ThrBundles[l][j]] for j in i+1:h) for i in 1:h-1) + t[ThrBundles[l][end]] * f[ThrBundles[l][end]]
)

optimize!(mod)
termination_status(mod)

LOCALLY_INFEASIBLE::TerminationStatusCode = 5

In [69]:
primal_feasibility_report(mod)

Dict{Any, Float64} with 1 entry:
  ((t[1] * f[1] * ((1.0 - f[3]) * (1.0 - f[4])) + t[3] * f[3] * … => 3.42728e-10

In [64]:
value.(t) |> println
value.(f) |> println

[2.9962542114803625, 4.315146043300834, 5.334970409723789, 9.707365021388634]
[0.8782556903051538, 0.3057100995525, 0.1903886995899347, 0.06853687639185227]


In [112]:
t = randexp(5) |> sort
f = rand(5)

function valuation(x; t=t, f=f)
    m = length(x)
    return sum(x[i] * t[i] * f[i] * prod(1 - f[j]*x[j] for j in i+1:m) for i in 1:m-1) + x[m] * t[m] * f[m]
end

hessian(valuation, [0, 1, 0, 1, 1]) |> eigen

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
5-element Vector{Float64}:
 -0.6203265849350809
 -6.793801715188064e-5
  0.011757297401161937
  0.024919951011714625
  0.5837172745393578
vectors:
5×5 Matrix{Float64}:
 -0.0180513   0.993189     0.107049  -0.0403568    0.0126785
 -0.276346    0.0836279   -0.573324   0.732323     0.227247
 -0.267013    0.0345161   -0.647114  -0.679051     0.218282
 -0.695493   -0.00864249   0.046988  -0.00220121  -0.71694
 -0.606878   -0.0729043    0.488749  -0.0309784    0.62173